# Pich detection using SPICE model

メモ: あとでmusic21, pydubのインストールをしておくこと

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display as librosadisplay

import logging
import math
import statistics
import sys

from IPython.display import Audio, Javascript
from scipy.io import wavfile

from base64 import b64decode

import music21
from pydub import AudioSegment

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

print(f'tensorflow: {tf.__verson__}')



In [ ]:
# JS code
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({audio: true})
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
    }
    recorder.stop()
})
"""

In [ ]:
def record(sec=5):
    try:
        from google.colab import output
    except ImportError:
        print('no possible to import output from google.colab')
        return
    else:
        print('Recording')
        display(Javascript(RECORD))
        s = output.eval_js('record(%d)' % (sec*1000))
        fname = 'recorded_audio.wav'
        print('Saving to', fname)
        b = b64decode(s.split(',')[1])
        with open(fname, 'wb') as f:
            f.write(b)
        return fname

### Input your audio

In [ ]:
INPUT_SOURCE = 'https://storage.googleapis.com/download.tensorflow.org/data/c-scale-metronome.wav'

print(f'You selected {INPUT_SOURCE}')

if INPUT_SOURCE == 'RECORD':
    uploaded_file_name = record(5)
elif INPUT_SOURCE == 'UPLOAD':
    try:
        from google.colab import files
    except ImportError:
        print("ImportError: files from google.colab seems to not be available")
    else:
        uploaded = files.upload()
        for fn in uploaded.keys():
            print(f'User uploaded file {fn} with length {len(uploaded[fn])} bytes')
        uploaded_file_name = next(iter(uploaded))
        print(f'Uploaded file: {uploaded_file_name}')

elif INPUT_SOURCE.startswith('./drive/'):
    try:
        from google.colab import drive
    except ImportError:
        print("ImportError: files from google.colab seems to not be available")
    else:
        drive.mount('/content/drive')
    # don't forget to change the name of the file you will you here!
    gdrive_audio_file = 'your audio file here'
    uploaded_file_Name = INPUT_SOURCE
elif INPUT_SOURCE.startswith('http'):
    # wget --no-check-certificate 'https://storage.googleapis.com/download.tensorflow.org/data/c-scale-metronome.wav' -O c-scale.wav
    uploaded_file_name = 'c-scale.wav'
else:
    print('Unrecognized input format!')
    print('Please select "RECORD", "UPLOAD", or specify a file hosted on Google Drive or a file from the web to download file to download')
